## Log returns and Simple returns

Insight: If innovations in log returns are iid then, log returns at long enough horizons must be normal (Central Limit Theorem)

$$1+𝑅_{1→𝑇}=((1+𝑅_1 )(1+𝑅_2 )…(1+𝑅_𝑇))⁡\\
ln⁡(1+𝑅_{1→𝑇} )=ln⁡((1+𝑅_1 )(1+𝑅_2 )…(1+𝑅_𝑇 ))\\
ln⁡(1+𝑅_{1→𝑇} )=ln⁡(1+𝑅_1 )+ln⁡(1+𝑅_2 )+…+ln⁡(1+𝑅_𝑇 )\\
                    𝑟_{1→𝑇}=𝑟_1+𝑟_2+…+𝑟_𝑇 $$ 



* Thus, if T is large, $𝑟_{1→𝑇}$ will be approximately normally distributed. (central limit theorem)

* $𝑟_𝑡=ln⁡(1+𝑅_𝑡 )$ is the rate of return assuming continuous compounding for the period t.

* This turns out to hold up well if you look at horizons longer than a month, but does not work at all at daily frequency (much fatter left tail than the normal distribution predicts)


* Log returns are also very convenient when thinking about long-term investing

In [33]:
print([np.log(1+Data.MKT).skew(),X.skew()])

print([np.log(1+Data.MKT).kurtosis(),X.kurtosis()])

[-0.7707864524331097, -0.10212480191113225]
[2.859221702344219, -0.17043746558020745]


#### How to go back and fourth between log and simple returns?

Let R be a simple net return per period, i.e., if you invest 10 , you get 10(1+R) in the end of the period

R is a number like 5% (0.05)

We say 1+R, a number like 1.05 is a gross return. 

Why gross? Because it includes 1, your initial investment.

* To get log returns: $r=log(1+R)$


* To get back simple returns: $R=exp(r)-1$

In [34]:
r=np.log(1+Data.MKT)
R=np.exp(r)-1
pd.concat([R,Data.MKT],axis=1).head()

,MKT,MKT
Date,,
1963-02-28,-0.0215,-0.0215
1963-03-31,0.0331,0.0331
1963-04-30,0.0476,0.0476
1963-05-31,0.0200,0.0200
1963-06-30,-0.0177,-0.0177


In [35]:
threshold=3
# now real data in log returns

A=((np.log(Data.MKT+1)-np.log(Data.MKT+1).mean())<-threshold*np.log(Data.MKT+1).std())
B=((X-X.mean())<-threshold*X.std())
[A.sum(),B.sum()]

[4, 0]

### The tangency portfolio and the cross-sectional of expected returns


* The optimal condition for our tangency portfolio:

$$ Var(R^e)W^*=cte \times E[R^e]$$

* This condition above defines the tangency portfolio

* We can rewirte the left hand side as

$$Var(R^e)W^*=Cov(R^e,R^e)W^*=Cov(R^e,(W^*)'R^e)=Cov(R^e,R^*)$$

* The left-hand side in the covariance vector of all the assets with the tangency portfolio

* This means that the expected return of an asset must be proportional to the covariance between the return of the asset and the return of the tangency portfolio

$$ E[R^e]=cte \times Cov(R^e,R^*) $$


* This holds for all assets in your investment opportunity set (i.e. the assets included in the minimization problem that the tangency portfolio solves)


* This means that for any assets $j$ and $i$ we have 


$$\frac{E[R^e_i]}{Cov(R^e_i,R^*)}= \frac{E[R^e_j]}{Cov(R^e_j,R^*)} $$ 


* All assets have the same expected return per-unit of covariance with the tangency portoflio

* We can also apply this to the tangency portfolio itself


$$ E[R^*]=cte \times Cov(R^*,R^*) $$


* Which means that $cte=\frac{E[R^*]}{Var(R^*)}$


* Plugging back in the original and we get that for any asset $j$


$$ E[R^e_j]=\frac{E[R^*]}{Var(R^*)} Cov(R^e_j,R^*) =\frac{Cov(R^e_j,R^*)}{Var(R^*)}E[R^*]=\beta_{j,*}E[R^*] $$



$$ E[R^e_j]=\beta_{j,*}E[R^*] $$

> What does that remind you?

* All the information about the cross-sectional of expected returns--all the assets-- can be summarized by the expected return of the tangency portfolio and the beta of each asset with respect to the tangency portfolio

* Or Alternatively: the risk-return trade-off  of the tangency portfolio  $\frac{E[R^*]}{Var(R^*)}$ and the covariance of each asset with it.


* Once you have the right notion of risk, the reward to risk is the same across assets (no free lunch)

* What is the right notion? Covariance with the tangency portfolio

* In the sample that you estimated the moments this will be true by construction!

Now we test if $ E[R^e_j]=\frac{Cov(R^e_j,R^*)}{Var(R^*)}E[R^*] $ in our sample

In [ ]:
Wstar

In [ ]:
Rstar=Re.copy()
Rstar['*']=Re @ Wstar
Rstar.cov()

In [ ]:
cov=Wstar @ Re.cov()
cov

In [ ]:
# expected return of the tangency portfolio
mu_star=Wstar @ Re.mean()
# variance of the tangency portfolio
var_star=(Wstar @ Re.cov() @ Wstar.T)
# covariance between each asset return and the tangency portfolio
cov=Wstar @ Re.cov()

predicted= mu_star/var_star*cov

print(predicted)

print(Re.mean())

* This comes from the fact that the tangency portfolio was constructed to be the portfolio with the maximum Sharpe Ratio
* It is a result of us knowing the moments of all assets summarized by $E[R^e]$  and $Var(R^e)$
* Here we did everything IN SAMPLE. We used sample moments and evalauted it's predictions using the same sample
* The challenge is that these moments are not always well estiamted, to put it mildly (more on this soon!)
* So the in sample tangency portfolio might not be very informative about the true tangency portfolio, which is the one that we actually care about

> Why? Because it can predict returns going forward and not only in sample (which is useless)